In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from skimage.exposure import rescale_intensity
from sklearn.metrics.cluster import entropy
import seaborn as sns

In [4]:
class HSI_Feauture_Analysis:
    
    def __init__(self, angle=90):
        
        self.angle = angle # угол (90 или 45)
        self.days = [1, 6, 8, 12, 19, 25] # список используемых дней
        self.name_indeces = ['TIR', 'NDVI', 'GNDVI', 'BLUE', 'GCL', 'SIPI'] # список имен используемых индексов
        self.name_features = ['Max', 'Min', 'Max - Min', 'Mean', 'Std'] # список имен используемых стат.признаков
        self.n_split = 10 # кол-во разбиений каждого изображения
        self.count_image_for_day = 2 # кол-во изображений, взятых с каждого дня
        
        # Рассматриваем 2 изображения с каждого дня эксперимента. 
        if angle == 90:
            days_left = [(25, 1), (8, 1)] # (день, номер) выбранных изображений левых-влажных частей (т.е. 1 дня)
            days_right = [(6, 1), (6, 3), (8, 2), (8, 3), (12, 1), (12, 3), (19, 1), (19, 3), (25, 1), (25, 3)] # правых-сухих
            self.path = 'TIR + HSI 90'
        elif angle == 45:
            days_left = [(25, 1), (25, 2)] 
            days_right = [(6, 2), (6, 3), (8, 2), (8, 3), (12, 1), (12, 3), (19, 1), (19, 3), (25, 1), (25, 3)]
            self.path = 'TIR + HSI 45'
            
        self.days_all = days_left + days_right
        
        pass
    
    def Percentile_Crop(self, img, q1=5, q2=95):
        # Зануляет выбросы у изображения (img) по процентилям (q1 и q2, где q1 < q2)
        res = img.copy()
        p1, p2 = np.percentile(res[res != 0], [q1, q2])
        res[(res > p2) | (res < p1)] = 0
        return res
    
    def NDVI(self, hsi):
        # Вычисление индекса NDVI = (NIR – RED) / (NIR + RED)
        RED = (hsi[:, :, 96] + hsi[:, :, 97]) / 2
        NIR = (hsi[:, :, 136] + hsi[:, :, 137]) / 2

        num = NIR - RED
        denum = NIR + RED
        denum[denum == 0] = 1
        ndvi = num / denum
    
        return self.Percentile_Crop(ndvi)
    
    def GNDVI(self, hsi):
        # Вычисление индекса GNDVI = (NIR – Green) / (NIR + Green)
        GREEN = hsi[:, :, 54]
        NIR = (hsi[:, :, 136] + hsi[:, :, 137]) / 2
    
        num = NIR - GREEN
        denum = NIR + GREEN
        denum[denum == 0] = 1
        gndvi = num / denum
    
        return self.Percentile_Crop(gndvi)
    
    def BLUE(self, hsi):
        # Извлечение синего канала
        BLUE = (hsi[:, :, 28] + hsi[:, :, 29]) / 2
        return self.Percentile_Crop(BLUE)
    
    def GCL(self, hsi):
        # Вычисление индекса GCL = NIR / GREEN - 1
        GREEN = hsi[:, :, 54]
        NIR = (hsi[:, :, 136] + hsi[:, :, 137]) / 2 
    
        num = NIR
        denum = GREEN
        gcl = np.zeros(hsi.shape[:2])
        gcl[denum != 0] = num[denum != 0] / denum[denum != 0] - 1
    
        return self.Percentile_Crop(gcl)
    
    def SIPI(self, hsi):
        # Вычисление индекса SIPI = (NIR - BLUE) / (NIR - RED)
        BLUE = (hsi[:, :, 28] + hsi[:, :, 29]) / 2
        RED = (hsi[:, :, 96] + hsi[:, :, 97]) / 2
        NIR = (hsi[:, :, 136] + hsi[:, :, 137]) / 2
    
        num = NIR - BLUE
        denum = NIR - RED
        sipi = np.zeros(hsi.shape[:2])
        sipi[(num != 0) & (denum != 0)] = num[(num != 0) & (denum != 0)] / denum[(num != 0) & (denum != 0)]
    
        return self.Percentile_Crop(sipi)
    
    def ARVI(self, hsi):
        # Вычисление индекса ARVI = (NIR – (2 * Red) + Blue) / (NIR + (2 * Red) + Blue)
        BLUE = (hsi[:, :, 28] + hsi[:, :, 29]) / 2
        RED = (hsi[:, :, 96] + hsi[:, :, 97]) / 2
        NIR = (hsi[:, :, 136] + hsi[:, :, 137]) / 2
        
        num = NIR - 2 * RED + BLUE
        denum = NIR + 2 * RED + BLUE
        denum[denum == 0] = 1
        arvi = num / denum
    
        return self.Percentile_Crop(arvi)
    
    def Blue_Green_Index(self, hsi):
        # Вычисление индекса BGI = (Green - Blue) / (Green + Blue)
        BLUE = (hsi[:,:, 28] + hsi[:, :, 29]) / 2
        GREEN = hsi[:, :, 54]
        
        num = GREEN - BLUE
        denum = GREEN + BLUE
        denum[denum == 0] = 1
        bgi = num / denum
        
        return self.Percentile_Crop(bgi)
    
    def Feature_Extraction(self, img, stat_features=[np.max, np.min, lambda x: np.max(x) - np.min(x), np.mean, np.std]):
        # Извлечение статистических признаков из одноканального изображения (img) 
        res = img[img != 0]
        return np.array([func(res) for func in stat_features])
        #t_max, t_min = res.max(), res.min()
    
        #return np.array([t_max, t_min, t_max - t_min, res.mean(), res.std()])
        
    def Separating_Index(self, hsi, thr=0.2):
        # Вычисление спектрального индекса, отделяющего растение от фона по порогу thr
        # Возвращает индекс и hsi с выделенным растением
        
        si = np.abs(hsi[:, :, 52:54].mean(axis=2) - hsi[:, :, 18])
        si /= (si.max() - si.min())
        si[si > 1] = 1
        si[si < thr] = 0
        
        hsi_wheat = hsi.copy()
        hsi_wheat[si == 0] = 0
        return si, hsi_wheat
        
    def Separating_NDVI(self, hsi, thr = 0.1):
        # Аналог Separating_Index, отделение растения по NDVI
        # Возвращает индекс и hsi с выделенным растением
        
        ndvi = self.NDVI(hsi)
        hsi_wheat = hsi.copy()
        hsi_wheat[ndvi < thr] = 0
        return ndvi, hsi_wheat
    
    def Plot_HSI(self, hsi):
        # Отображение HSI в виде псевдо RGB
        
        fig, ax = plt.subplots(figsize=(5, 5))
        ax.axis('off')
        
        p_rgb = self.hsi[:, :, (70, 53, 19)] # Псевдо-rgb
        p_rgb = rescale_intensity(p_rgb, in_range=(0,50), out_range=(0,255)) # увеличение яркости
        plt.imshow(p_rgb)
        print(f'Размер HSI: {self.hsi.shape}')
        
    def Plot_TIR(self, tir, t_min=0, t_max=0, vi_flag=False, name='TIR'):
        # Отображение TIR (либо ВИ при vi_flag=True) в псевдоцветах и гистограммы данных
        
        tir_line = tir[tir != 0]
        if t_min == 0 and t_max == 0: t_min, t_max = tir_line.min(), tir_line.max()
    
        fig, ax = plt.subplots(1, 2, figsize=(9, 5), constrained_layout=True)
        
        # Построение цветовой карты
        color_list = ["lawngreen", "yellow", "red"]
        if vi_flag: color_list.reverse()
        tir_map = LinearSegmentedColormap.from_list("mycmap", color_list)
        tir_map.set_under('black')
        
        tir_pic = ax[0].imshow(tir, cmap=tir_map, vmin=t_min, vmax=t_max)
        fig.colorbar(tir_pic, ax=ax[0])
        ax[1].hist(tir_line, 500, [t_min, t_max])
        ax[0].axis('off')
        ax[0].set_title(name, fontsize=15)
        ax[1].set_title(f'Гистограмма {name}', fontsize=15)
        plt.show()
        print(f'Размер: {tir.shape}, Минимум: {tir_line.min()}, Максимум: {tir_line.max()}')
        
    def Plot_Group_Image(self, images_lst, names_lst, shape_img=(3, 2)):
        # Отображение нескольких изображений рядом
        
        fig, ax = plt.subplots(shape_img[0], 2 * shape_img[1], figsize=(16, 9), constrained_layout=True)
        
        # Построение цветовой карты
        color_list = ["lawngreen", "yellow", "red"]
        
        tir_map = LinearSegmentedColormap.from_list("mycmap", color_list) # Для TIR
        tir_map.set_under('black')
        
        tir_map_rev = LinearSegmentedColormap.from_list("mycmap", color_list[::-1]) # Для ВИ
        tir_map_rev.set_under('black')
        
        for i in range(shape_img[0]):
            for j in range(0, 2 * shape_img[1], 2):
                
                cur_pic = images_lst[i * shape_img[1] + j // 2]
                cur_data_non_zero = cur_pic[cur_pic != 0]
                cur_min, cur_max = cur_data_non_zero.min(), cur_data_non_zero.max()
                
                cur_graph = ax[i, j].imshow(cur_pic, cmap=(tir_map if i==j==0 else tir_map_rev), vmin=cur_min, vmax=cur_max)
                fig.colorbar(cur_graph, ax=ax[i, j])
                ax[i, j].axis('off')
                ax[i, j].set_title(names_lst[i * shape_img[1] + j // 2], fontsize=15)
                
                ax[i, j + 1].hist(cur_data_non_zero, 500, [cur_min, cur_max])
                ax[i, j + 1].set_title(f'Гистограмма {names_lst[i * shape_img[1] + j // 2 ]}', fontsize=15)
                
        plt.show()
                
        
    
    def Load_Data(self, path='TIR + HSI 90', day=6, num=1, thr=0.2):
        # Загрузка HSI и TIR; Отделение фона от растения
        
        mask, hsi = self.Separating_Index(np.load(f'{path}/{day}/HSI_{num}.npy'), 0.2)
        tir = np.load(f'{path}/{day}/TIR_{num}.npy')
        tir[mask == 0] = 0
        
        return hsi, tir
    
    def Half_Image(self, img, side='right'):
        # Оставляет от изображения левую-влажную часть (side='left') или правую-сухую (side='right')
        
        if side == 'right':
            res = img[:, img.shape[1] // 2:].copy()
        elif side == 'left':
            res = img[:, :img.shape[1] // 2].copy()
        
        return res
    
    def Split_Image(self, img, n_split=10):
        # На вход дается изображение (TIR)
        # Пиксели содержащие пшеницу перемешиваются и делятся на n_split частей
        # Возвращает двумерный список размера (n_split, num), но последняя строка имеет размер <= num
        
        len_img = len(img[img != 0]) # кол-во пикселей пшеницы на изображении
        num = len_img // n_split # кол-во пикселей в одной части
        indeces = np.array(range(len_img))
        np.random.shuffle(indeces)
        
        return [indeces[num * k : num * k + num] for k in range(n_split)]
    
    def Main_Process(self, vis_flag=True):
        # Главная функция, в процессе которой создается словарь признаков
        # vis_flag - нужно ли визуализировать TIR, ВИ и гистограммы (визуализация увеличивает время работы программы)
        
        days_lst = np.repeat(self.days, self.count_image_for_day)
        
        indeces_lst = [self.NDVI, self.GNDVI, self.BLUE, self.GCL, self.SIPI] # выбранные для анализа ВИ
        stat_features = [np.max, np.min, lambda x: np.max(x) - np.min(x), np.mean, np.std] # выбранные стат. признаки
        
        
        F_Dict = Dict_Features(name_indeces=self.name_indeces, name_features=self.name_features) # Создание словаря признаков
        
        for i, el in enumerate(self.days_all):
            
            # Загрузка HSI, TIR
            hsi, tir = self.Load_Data(path=self.path, day=el[0], num=el[1], thr=0.2) 
            
            # Отделение половины HSI (левой-влажной для 1 дня, правой-сухой для остальных дней)
            hsi = self.Half_Image(hsi, side=('left' if i < 2 else 'right')) 
            
            # Отделение половины TIR, удаление выбросов
            tir = self.Percentile_Crop(self.Half_Image(tir, side=('left' if i < 2 else 'right')), q1=5, q2=95) 
            
            # Вычисление ВИ, кладем их в список
            images_lst = [indx(hsi) for indx in indeces_lst] 
            del hsi
            
            
            if vis_flag:
                # Визуализация TIR, ВИ, их гистограмм
                print('-' * 120 + f'\n День {days_lst[i]}.{i % 2 + 1}')
                self.Plot_Group_Image(images_lst=[tir] + images_lst, names_lst=self.name_indeces, shape_img=(3, 2))
            
            # Разделение TIR и ВИ на случайные части, добавление стат.признаков этих частей в словарь
            indeces = self.Split_Image(tir, n_split=self.n_split) # Разделение TIR на n_split=10 частей
            for i in range(len(indeces)):
                x = indeces[i] # Список пикселей текущей части
                half_tir = self.Feature_Extraction(tir[tir != 0][x], stat_features=stat_features) # Стат.признаки TIR
                half_VI = [self.Feature_Extraction(indx[tir!= 0][x], stat_features=stat_features) 
                           for indx in images_lst] # Список стат.признаков ВИ
                
                
                F_Dict.Add_All_Features([half_tir] + half_VI) # Добавление стат.признаков в словарь признаков
                
        return F_Dict.Take_Dict()
    
    
    def Plot_Feature(self, dct):
        # Визуализация стат.признаков из словаря признаков
        
        multiplier = self.count_image_for_day * self.n_split
        
        fig, ax = plt.subplots(len(self.name_indeces), len(self.name_features), figsize=(27, 19))
        for i in range(len(self.name_indeces)):
            for j in range(len(self.name_features)):
                
                # Построение точек значений признаков
                x_data = np.repeat(self.days, multiplier)
                y_data = dct[self.name_indeces[i]][self.name_features[j]]
                sns.scatterplot(x=x_data, y=y_data, ax=ax[i, j], color='blue', label="value")
                
                # Построение средней линии
                y_data_mean = np.array(y_data).reshape(len(self.days), multiplier).mean(axis=1)
                sns.lineplot(x=self.days, y=y_data_mean, ax=ax[i, j], color='black', label="mean")
                
                # Подписи к графикам
                ax[i, j].set_title(f'{self.name_indeces[i]} {self.name_features[j]}', fontsize=15, color='blue')
                ax[i, j].set_xlabel('Day', fontsize=11)
                ax[i, j].set_ylabel('Value', fontsize=11)
                ax[i, j].legend()
        
        plt.subplots_adjust(wspace=0.3, hspace=0.5) # расстояние между графиками
        plt.show()
                
    def Save_Feature(self, dct, path_out='Table'):
        # Сохранение признаков в таблицу
        
        for name_ind in self.name_indeces:
            
            table = {**{'Day' : np.repeat(self.days, self.count_image_for_day * self.n_split)}, 
                     **{name_f : dct[name_ind][name_f] for name_f in self.name_features}}
            
            data = pd.DataFrame.from_dict(table)
            
            path_lst = [path_out, str(self.angle)]
            Create_Folder(path_lst[0])
            Create_Folder('/'.join(path_lst))
            data.to_excel(f'{"/".join(path_lst)}/{name_ind}_Stat.xlsx')